In [1]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import requests
from textwrap import shorten
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_csv('PlannedCondosWPB - Sheet1.csv')

In [3]:
df.columns

Index(['Developer', 'Name ', 'Site Address', 'Condo/Rental', 'Units',
       'Stories', 'Price Range', 'Floorplans', 'SF', 'Brokerage',
       'Sales Launch', 'Est. Completion', 'Dev Site Cost', 'Link'],
      dtype='object')

## Geocode

In [4]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [5]:
df['Site Address'] = df['Site Address'] + ' West Palm Beach, FL'

In [6]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Site Address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Link Grabber and Image Snagger

In [10]:
def popup_html(row_index):
    # Initialize the HTML content with a <style> tag to change the font
    html_content = '''
    <style>
        .popup-content {
            font-family: Arial, sans-serif; /* Change the font family as needed */
            font-size: 14px;
        }
        img {
            max-width: 100%; /* Ensure images are not too wide */
            height: auto; /* Maintain aspect ratio */
            margin-top: 5px; /* Add some space above the image */
        }
    </style>
    <div class="popup-content">
    '''

    # Dynamically add data for each column that has a value
    for column in df.columns:
        value = df.at[row_index, column]
        value_str = str(value).strip()
        if pd.notnull(value) and value_str != "":
            if column == 'image_links' and value_str != "No Image":  # Assuming 'No Image' indicates absence of an image
                # Embed the image using an <img> tag
                html_content += f'<img src="{value_str}" alt="Image"><br>'
            elif column not in ['lat', 'lon', 'Link', 'image_links']:  # Exclude certain columns from the text part
                html_content += f'<strong>{column}:</strong> {value_str}<br>'

    # Close the div tag
    html_content += '</div>'

    return html_content


m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=12)


for i in range(len(df)):
    popup_content = popup_html(i)
    iframe = folium.IFrame(html=popup_content, width=300, height=150)  # Adjust iframe size as needed
    popup = folium.Popup(iframe, max_width=350)  # Adjust max_width as needed
    folium.Marker(
        location=[df.iloc[i]['lat'], df.iloc[i]['lon']],
        popup=popup
    ).add_to(m)
    
m

In [9]:
m.save('index.html')